# Import Libraries

In [1]:
!pip install Sastrawi scikit-learn
!pip install --upgrade nbformat
!pip install nltk


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
from google_play_scraper import Sort, reviews
import pandas as pd

import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

#from nltk.corpus import stopwords

from joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kaylaalysa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/kaylaalysa/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Load Dataset

In [3]:
df  = pd.read_csv('reviews_shopee.csv')

In [4]:
df.head()

,review,rating
0,Belanja mudah.. ya di shoope 😁,4
1,"UI nya jelek, proteksi penipuan sangat buruk. ...",1
2,Sangat membantu... tingkat kan lagi shoope,5
3,Aplikasi belanja ter okk,5
4,bagus,5


In [5]:
len(df)

10000

# Preprocessing Dataset

In [6]:
# Cek missing values dan duplicated data
print('Jumlah missing values = ', df.isnull().sum().sum())
print('Jumlah duplicated values = ', df.duplicated().sum().sum())

Jumlah missing values =  0
Jumlah duplicated values =  2279


In [7]:
# hapus data duplikat
df.drop_duplicates()

,review,rating
0,Belanja mudah.. ya di shoope 😁,4
1,"UI nya jelek, proteksi penipuan sangat buruk. ...",1
2,Sangat membantu... tingkat kan lagi shoope,5
3,Aplikasi belanja ter okk,5
4,bagus,5
...,...,...
9994,"Sopi emang bisa di percaya,🙏🙏",5
9995,"Pelayanan bagus,kualitas barang ok",5
9996,Hmm menurut saya bagus tapi ada kaya bug bug gitu,1
9998,Aplikasi ini sangat bagus😊👍,5


In [8]:
# casefolding
def case_folding(review):
    review = review.lower()
    return review

df['review'] = df['review'].apply(case_folding)
df.head(10)

,review,rating
0,belanja mudah.. ya di shoope 😁,4
1,"ui nya jelek, proteksi penipuan sangat buruk. ...",1
2,sangat membantu... tingkat kan lagi shoope,5
3,aplikasi belanja ter okk,5
4,bagus,5
5,shopee bagus produk2nya harga pun relatif mura...,2
6,sangat membantu. mudah,5
7,sangat bermanfaat,4
8,pokoknya top deh,5
9,bagus,5


In [9]:
# cleaning data
def clean_data(review):
    review = review.strip(" ")
    review = re.sub(r'@[A-Za-z0-9]+', '', review)
    review = re.sub(r'#[A-Za-z0-9]+', '', review) 
    review = re.sub(r'RT[\s]', '', review) 
    review = re.sub(r"http\S+", '', review) 
    review = re.sub(r'[0-9]+', '', review) 
    review = re.sub(r'[^\w\s]', '', review)
    review = review.replace('\n', ' ') 
    review = review.translate(str.maketrans('', '', string.punctuation)) 
    review = review.strip(' ') 
    return review

df['review'] = df['review'].apply(clean_data)
df.head(10)

,review,rating
0,belanja mudah ya di shoope,4
1,ui nya jelek proteksi penipuan sangat buruk cu...,1
2,sangat membantu tingkat kan lagi shoope,5
3,aplikasi belanja ter okk,5
4,bagus,5
5,shopee bagus produknya harga pun relatif murah...,2
6,sangat membantu mudah,5
7,sangat bermanfaat,4
8,pokoknya top deh,5
9,bagus,5


In [10]:
#tokenized
def word_tokenized(text):
    return word_tokenize(text)

df['review'] = df['review'].apply(word_tokenized)
df.head(10)

,review,rating
0,"[belanja, mudah, ya, di, shoope]",4
1,"[ui, nya, jelek, proteksi, penipuan, sangat, b...",1
2,"[sangat, membantu, tingkat, kan, lagi, shoope]",5
3,"[aplikasi, belanja, ter, okk]",5
4,[bagus],5
5,"[shopee, bagus, produknya, harga, pun, relatif...",2
6,"[sangat, membantu, mudah]",5
7,"[sangat, bermanfaat]",4
8,"[pokoknya, top, deh]",5
9,[bagus],5


In [11]:
def remove_stopwords(text):
    indonesian_stopwords = set(stopwords.words('indonesian'))
    english_stopwords = set(stopwords.words('english'))
    
    additional_stopwords = {'iya', 'yaa', 'gak', 'nya', 'na', 'sih', 'ku', 'di', 'ga', 'ya', 'gaa', 'loh', 'kah', 'woi', 'woii', 'woy', 'yang', 'dari', 'untuk', 'dan', 'di', 'ke', 'dengan', 'pada', 'ini', 'itu', 'atau', 'sebagai', 'adalah'}
    
    all_stopwords = indonesian_stopwords.union(english_stopwords).union(additional_stopwords)
    
    filtered_text = [word for word in text if word not in all_stopwords]
    
    return filtered_text

df['review'] = df['review'].apply(remove_stopwords)
df.head(10)

,review,rating
0,"[belanja, mudah, shoope]",4
1,"[ui, jelek, proteksi, penipuan, buruk, custome...",1
2,"[membantu, tingkat, shoope]",5
3,"[aplikasi, belanja, ter, okk]",5
4,[bagus],5
5,"[shopee, bagus, produknya, harga, relatif, mur...",2
6,"[membantu, mudah]",5
7,[bermanfaat],4
8,"[pokoknya, top, deh]",5
9,[bagus],5


In [12]:
def apply_stemming(review):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in review:
        dt = stemmer.stem(w)
        do.append(dt)
    data_clean=[]
    data_clean=" ".join(do)
    print(data_clean)
    return data_clean

df['review'] = df['review'].apply(apply_stemming)
df.head(10)

belanja mudah shoope
ui jelek proteksi tipu buruk customer service bantu kasih
bantu tingkat shoope
aplikasi belanja ter okk
bagus
shopee bagus produk harga relatif murah tapiii ekspedisi spx buruk paket toko ekspedisi bertanggungjawab spx
bantu mudah
manfaat
pokok top deh
bagus
diskon ongkir
shpee jamin balang pesan sesuai
bagus banget
mantap
aplikasi belanja andal hehe
mantapshopeepayy
paylater spinjam syarat syarat mohon bantu
sorry shopee kali sungguh kecewa layan
shopee mantap
reel pokoke best jual msh seger pohon moga cepat besarsubur paccingnya keren
masuk shopee susah banget scan kode masuk aja blum udah suruh scan kode sulit udah pake hp
kesini isi saldo udh kena biaya admin eh aja potong aplikasi bayar pake shopee pay saldo ttp
bayar kali verifikasi bayar mesan barang chat cs shopee bilang sabar aja
shopee saing toko online papan kembang fitur mudah langgan terimakasih
g login lg
aplikasi shopee yg bagus pokok puas pakai turut aplikasi baik
mantab oke
tonton kecewa maaf hapus

shopee mmng best
best
krna kirim cepat
good job
bagus
puas belanja shopee
puas layan
kembali barang dana tanggal yg tentu shoppe tinjau mulu heran gw

baguuss guss gusss
good
good shopee family
ok
ribetsangat membsntu
marketplace jual beli online utama cocok banget beli apa marketplace kirim cepat pesan cocok banget barangbarang promo muncul recommended banget
ok
sdh update tp log
bantu belanja trimakasih shopee moga berkat
login
koin suka ngurangin
mantap poko
belanja online baik kualitas shopee
belanja mudah murah mangtabs
mantap

scam
kesini ribet saldo sophepay ribu tp check barang total rb gk bsa ngeselin pakai promo ca rb tetep gk bsa kacauuuu
kesini aplikasi jelek suka ngelag loadingnya banget tolong baik shopee aplikasi belanja online favorit
gimana cerita nyaditoko harga barang k pas masuk keranjang k
login pake metode udah pake hp pake link google fb proses udah selesai pas coba liat akun diauruh pilih opsi daftar login aga ganggu bugnya mending pake aplikasi
sukses
shope mud

apk bagus grts ongkir
gajelas notif bikin device ngelek

kirim cepat aman
kirim spx salah harga udah apk
senang
iklan shoope orang kagak download buka kagak yg mencet anj
ayo belanja toko bolt nut toko baut jamin aman
keluh yg bikin bingung tokoh yg gk respon chat gimana beli paket kurir balikin toko alas gk telpon gk pulsa tu kurir shopee pleaselah shopee rekrut kurir shopee ahli cari alamat coba deh telepon pakai telepon pakai wa trus dana proses tu toko alas barang yg batal gk dibalikin lgi parahhhhh
zx ae
mohon tingkat paylater
suka duka
barang bagus harga murah
briliant
langgan shopeee siiip banget
bantu belanja
tarik
baik
jujur tipu jual hp murah
nice
wihhh enk bnr duit shope pay akun ariny
aplikasi bagus butuh
harga murah meriahnyari aja
bantu
game masuk akal
paylater spinjam tolak min alas solusi gimana mimin shoope
bagusmbatu terima aksih
ayo cepat donglot shopi
ok bgt aplikasi belanja online yg percaya
bayar shopeepaylater pake shopeepay masalah udh hapus cache memori gin
mog

belanja senang rumah moga depan shopee jual yg amanahmengecewakan beli atas nama shopee
ok
suka
ok
saldo rb band bntu shope perbaikin terimakasi
godd jobs
mudah transaksi
shopeee bgus bgt suka aplikasi nyyy
good seller
bagus
jarang order shopeelebih kirim spx kurir kagak ngotakterutama daerah desa tata ulang pola rekrut kurir si cepat hilang pamor
mantab
buka aplikasi
tolong respon email gabisa login akun sendiridi hp uda logoutcoba masuk hp lg gabisaselalu notenya percobaa masukpadahal uda setujuin login akun gimanasaya uda coba beberspa kali gamau login saldo shope pay gaada respon shope kasi rate gaada responsaldo shope adarugi bgt kalo gabisa login lagianeh bgt
bagus
aja yg kirim slalu lambat emang gitu mohon tingkat kirim barang takut gara gara kirim telat aplikasi sepi minat
shopee bagus diskon gratis ongkir
semenjak shopi blanja onlen mudah shopi pay
aplikasi babi
baikk
kirim shopeexpres kali normal kirim daerah ekspedisi ubah tolong shopee kalo yg kota instan yg daerah masak nu

fitur pecah telur nggak nggak dpet koin lgi jarungan masalah
maaf ka semenjak pakai jasa spx barang lambat ujung kembali
sbb h ppoc
recommend
meebantu
aplikasi bagus sistem proteksi rusa
pokok shopee best
b berteletele
ekspedisi kaga jelasspx spx kerja
puas
ok sanggat shope pelanyanan
aplikasi bantu belanja
oke
sy belanja shopee puas produk yg ktinggalan dr pesan sy shopee kembali dana harga produk yg tinggal trsbut sy bahagia kenal shopee
moga shopee tingkat
guna
bantu
bagus
belanja mudah
jual sulit
produk yg sesuai dgn yg pes real pict
tks jempol
mudah murah ribet
kirim sen barang lokasi nyampe parah kirim shopee
woyyyyy pinjam eror limit geude pakeeeeeee
barang sesuai foto
aplikasi tipu warning nyesel banget kalo instal shopee tarik iklan shopee gratis ongkir bacot lo shopee udah beratberatin boit dasar bangat banget ni aplikasi
tolong baik system kurir semenjak pakai spx kirim kacau kembali kurir seblum jt jne
bagus
kecewa ama shope spinjam spaylater beku udah tagih cmn gara telat 

...

aplikator kejam semena mena lebih jajah jepang belanda pki
smakin fasilitas belanja dllsmakin selektif pilih jual yach shopee smakin ok
puas
mudah kalo belanja ribet
bantu aju pinjam spinjam
ubah status jual pesan barang non fisik telepon cs maksimal jam ubah jam ubah kalo gin kapok jual shopee daftar toko belah
mantap bantu semenjak shopi
rekomen banget
ok
fast respon layan bagus
shopee blnja jd ribet shopee amanah poko live shoping yuk harga murah dpt diskon gratis onkir voucher dll pnasaran trus dehh jdinya
siiiiip
kerja cs buruk
senang
puas
belanja murah ongkir murah
barang sesuai kirim sesuai jadwal
lumayan manfaat paylater
mantulll

baguss

bosan tunggu cepat sesuai yg janji
hai kk pake soopy kecewa sesuai bagus layananya n
apk ny bagus banget blnja online mudah praktis gratis ongkir mantap
oke
mantapppp
belanja hemat harga murah diskon raja rela
games bubble shopeemohon buka
ok
kecewa belanja gilir gratis ongkir belanja banget
sengaja instal aplikasi merating bintang aplikasi un

jelek lemottt bangettttt ngga nge refresh ngelag bener bener ngga ngotak
shopi oke
shopee later susah
always

muas belanja shoopy bnyk diskon barang kualitas respon cpt
kirim barang hemat standart beda kalok cepat
guna
aktif shopee
gagal gagal menu e walet jg
oke
kabar teman teman
shopie bantu
senang belanja shopee

muas
puas belanja shoppe
oke banget
muas

apk yg bantu hidup hari
wellll
shopee udah oke klw bole sihhh shopee ongkir kurang dikit
aplikasi bagus bagus promo voucher pokok top deh aplikasi shopee potong harga emak emak happy
shoppe biasabanyak manfaat tf bank lainterlebih lg aju pinjam sistim pesan bayar nantithe best
aplikasi sungguh bantu
bagus
tks shopee mudah belanja apa yg butuh cepat sampai
good aplikasi
thanks
shopee bantu hidup jaman sekarangbelanja mudah rumah belanja
ok
mantab
mantap deh
niceee
shopping nyaman percaya
kecewa berat voucher gua mas g dipake vhocer dipake gua blm mesan apa
bagus ak suka kerja
seller bela salah order bel
bagus tarik belanja
belanjamud

jual online bagus rekomenden
bagus
pakai shopy gampang mes yg butuh
mantap
shopee nih bagus bgt tapiiii ongkir luar sumatra lumayan mahal
ambil koin hasil nonton live gk shopee deteksi aktifitas curiga maksud gue yg lu curigain gitu
tolong kurir shopee baik lot banget
karna bagus aplikasi

bantu belanja onlin
belanja muas bantu banding olshop diskon gratis ongkir suka belanja shopee
kecewa tangan tim shopee ekspedisi lambat komplain suruh nunggu kirim paket tunggu paket geser hadeeeh
shopee pokok best
okeee tp sayang utk spinjam spaylaternya sy gk bsa pake pdhl tagih tunda lg

good
tks sopiee
layan kirim sesuwai komplen besok kirim males buruk
bisah hilanggin aja kirim shope expressnyakurirnya jelek kaya malas krja cuman daera yg jelek kirim kurir nyasemua daera kaya jelek kirim kurir
mantap shopee moga jaya
mantap aplikasi lancar jayaaa
spx burukpengiriman reguler lambat sesuai estimasipaket nyata hilang tanggung
suka shoppe
aplikasi bagus belanja
lambat ngistalnya
bagus
taiiiiiiiiiii

kadang suka responding
baguss
enak kaum rumah
jasa kirim spx sungguh lambat kali gk kayak jne jnt cepat kilat
mantsp
mantap
bagus
biar bintang bicara
shopee brengsek mentingin jual rugi beli bantu tdk perhati utk shopee brengsek

alhmdulillah mes brg shopee sya puasbarang sesuai dngan pesanankualitasnya bgus
mantap
bagusss
harap shopee depan fitur pilih kurir kurir gk ngantar paket rumah suruh ngambil paket gudang
pesan info shipment pesan kembali shopee barang tunggu

sumpah masuk akal shopee skrg kirim lelet
kirim banget gabisa pilih ekspedisi enak belanja shopee ekspedisi bikin badmood tolong ada fitur pilih ekspedisi biar nyaman
bagus
mantap pesan sesuai
tolong kurir pesan sesuai alamat yg batal pesan konfirmasi saya ajar kurir alamat antar
belanja shopee layan baik
barang yg kirim kadang beda
pinjem shoope pay semangat galbay
barang yg pes sesuai
muter aja pas beli sll tulis jasa kirim adamau cek ongkir ny brp muter aja nggak
hehey bad

baguss
bagus sesuai gambar
recomended banget

,review,rating
0,belanja mudah shoope,4
1,ui jelek proteksi tipu buruk customer service ...,1
2,bantu tingkat shoope,5
3,aplikasi belanja ter okk,5
4,bagus,5
5,shopee bagus produk harga relatif murah tapiii...,2
6,bantu mudah,5
7,manfaat,4
8,pokok top deh,5
9,bagus,5


In [13]:
slangwords = {"@": "di", "gak": "tidak", "wtb": "beli", "wts": "jual", "msh": "masih", "wtt": "tukar", "bgt": "banget", "maks": "maksimal",
             "moga":"semoga", "dlm":"dalam", "bgus":"bagus", "jlk":"jelek", "udah":"sudah", "utk":"untuk", "dg":"dengan", "apk":"aplikasi",
             "aq":"aku", "qm":"kamu", "yg":"yang"}


def fix_slangwords(text):
    words = text.split()
    fixed_words = []
 
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
 
    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [14]:
df['review'] = df['review'].apply(fix_slangwords)
df.head(10)

,review,rating
0,belanja mudah shoope,4
1,ui jelek proteksi tipu buruk customer service ...,1
2,bantu tingkat shoope,5
3,aplikasi belanja ter okk,5
4,bagus,5
5,shopee bagus produk harga relatif murah tapiii...,2
6,bantu mudah,5
7,manfaat,4
8,pokok top deh,5
9,bagus,5


# Labelling

In [25]:
def label_sentiment(rating):
    if rating >= 4:
        return 'Positif'
    elif rating == 3:
        return 'Netral'
    else:
        return 'Negatif'

In [26]:
df['label'] = df['rating'].apply(label_sentiment)

In [27]:
df

,review,rating,label
0,belanja mudah shoope,4,Positif
1,ui jelek proteksi tipu buruk customer service ...,1,Negatif
2,bantu tingkat shoope,5,Positif
3,aplikasi belanja ter okk,5,Positif
4,bagus,5,Positif
...,...,...,...
9995,layan baguskualitas barang ok,5,Positif
9996,hmm bagus kaya bug bug gitu,1,Negatif
9997,mantap,5,Positif
9998,aplikasi bagus,5,Positif


# Ekstraksi Fitur dengan TF IDF

In [28]:
indonesian_stopwords = stopwords.words('indonesian')
tfidf = TfidfVectorizer(max_features=5000, stop_words=indonesian_stopwords)

X_tfidf = tfidf.fit_transform(df['review']).toarray()

/opt/homebrew/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


# Encode Label ke angka
* Negatif = 0
* Netral = 1
* Positif = 2

In [29]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(df['label']) 

# Split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)

# Train

In [51]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 kelas: Positif, Netral, Negatif
])

In [52]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history  = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
250/250 [==============================] - 2s 5ms/step - loss: 0.5631 - accuracy: 0.7839 - val_loss: 0.4235 - val_accuracy: 0.7930
Epoch 2/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3754 - accuracy: 0.8801 - val_loss: 0.3963 - val_accuracy: 0.8570
Epoch 3/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3001 - accuracy: 0.9175 - val_loss: 0.4387 - val_accuracy: 0.8640
Epoch 4/10
250/250 [==============================] - 1s 4ms/step - loss: 0.2501 - accuracy: 0.9289 - val_loss: 0.4576 - val_accuracy: 0.8665
Epoch 5/10
250/250 [==============================] - 1s 4ms/step - loss: 0.2264 - accuracy: 0.9352 - val_loss: 0.4649 - val_accuracy: 0.8645
Epoch 6/10
250/250 [==============================] - 1s 4ms/step - loss: 0.2091 - accuracy: 0.9415 - val_loss: 0.5018 - val_accuracy: 0.8665
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 0.1887 - accuracy: 0.9449 - val_loss: 0.5297 - val_accuracy: 0.8655
Epoch 

In [53]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f'Akurasi data training: {train_accuracy * 100:.2f}%')

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Akurasi data testing: {test_accuracy * 100:.2f}%')

Akurasi data training: 96.99%
Akurasi data testing: 86.25%
